# Geographic Visualization of CoW vs UCDP/PRIO

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

## Import Country Shapes to GeoPandas DataFrame

In [2]:
cshapes_df = gpd.read_file("./Data/CShapes/cshapes_0.6/cshapes.shp")
cshapes_df.columns

Index(['CNTRY_NAME', 'AREA', 'CAPNAME', 'CAPLONG', 'CAPLAT', 'FEATUREID',
       'COWCODE', 'COWSYEAR', 'COWSMONTH', 'COWSDAY', 'COWEYEAR', 'COWEMONTH',
       'COWEDAY', 'GWCODE', 'GWSYEAR', 'GWSMONTH', 'GWSDAY', 'GWEYEAR',
       'GWEMONTH', 'GWEDAY', 'ISONAME', 'ISO1NUM', 'ISO1AL2', 'ISO1AL3',
       'geometry'],
      dtype='object')

In [3]:
cshapes_df.head()

,CNTRY_NAME,AREA,CAPNAME,CAPLONG,CAPLAT,FEATUREID,COWCODE,COWSYEAR,COWSMONTH,COWSDAY,...,GWSMONTH,GWSDAY,GWEYEAR,GWEMONTH,GWEDAY,ISONAME,ISO1NUM,ISO1AL2,ISO1AL3,geometry
0,Guyana,211982.004988,Georgetown,-58.2,6.800000,0,110,1966,5,26,...,5,26,2016,6,30,Guyana,328,GY,GUY,"POLYGON ((-58.17262 6.81222, -58.15494 6.82819..."
1,Suriname,145952.274029,Paramaribo,-55.2,5.833333,1,115,1975,11,25,...,11,25,2016,6,30,Suriname,740,SR,SUR,"POLYGON ((-55.12796 5.82217, -55.10445 5.83945..."
2,Trinidad and Tobago,5041.728952,Port-of-Spain,-61.5,10.650000,2,52,1962,8,31,...,8,31,2016,6,30,Trinidad and Tobago,780,TT,TTO,"MULTIPOLYGON (((-61.07945 10.82416, -61.07556 ..."
3,Venezuela,916782.217193,Caracas,-66.9,10.500000,3,101,1946,1,1,...,1,1,2016,6,30,Venezuela,862,VE,VEN,"MULTIPOLYGON (((-66.31029 10.62602, -66.28309 ..."
4,Samoa,2955.212366,Apia,-172.0,-13.800000,4,990,1976,12,15,...,1,1,2016,6,30,Samoa,882,WS,WSM,"MULTIPOLYGON (((-172.59650 -13.50911, -172.551..."


In [4]:
cow_geo_df = cshapes_df[cshapes_df['COWCODE'] != -1].drop(columns=['GWSYEAR', 'GWSMONTH', 'GWSDAY', 'GWEYEAR', 'GWEMONTH', 'GWEDAY', 'ISONAME', 'ISO1NUM', 'ISO1AL2'])
gw_geo_df = cshapes_df[cshapes_df['GWCODE'] != -1].drop(columns=['COWSYEAR', 'COWSMONTH', 'COWSDAY', 'COWEYEAR', 'COWEMONTH', 'COWEDAY','ISONAME', 'ISO1NUM', 'ISO1AL2'])

In [6]:
#cow_gdf = cshapes_df[cshapes_df['COWCODE'] != -1]
#cow_gdf = cow_gdf[['FEATUREID', 'geometry']]

#gw_gdf = cshapes_df[cshapes_df['GWCODE'] != -1]
#gw_gdf = gw_gdf[['FEATUREID', 'geometry']]

#cow_gdf.to_file("./Data/processed/cow_countries.geojson", driver='GeoJSON')
#gw_gdf.to_file("./Data/processed/gw_countries.geojson", driver='GeoJSON')

In [8]:
cow_geo_df['StartDate'] = cow_geo_df['COWSYEAR'].astype(str) + '-' + cow_geo_df['COWSMONTH'].astype(str) + '-' + cow_geo_df['COWSDAY'].astype(str)
cow_geo_df['StartDate'] = pd.to_datetime(cow_geo_df['StartDate'])

cow_geo_df['EndDate'] = cow_geo_df['COWEYEAR'].astype(str) + '-' + cow_geo_df['COWEMONTH'].astype(str) + '-' + cow_geo_df['COWEDAY'].astype(str)
cow_geo_df['EndDate'] = pd.to_datetime(cow_geo_df['EndDate'])

gw_geo_df['StartDate'] = gw_geo_df['GWSYEAR'].astype(str) + '-' + gw_geo_df['GWSMONTH'].astype(str) + '-' + gw_geo_df['GWSDAY'].astype(str)
gw_geo_df['StartDate'] = pd.to_datetime(gw_geo_df['StartDate'])

gw_geo_df['EndDate'] = gw_geo_df['GWEYEAR'].astype(str) + '-' + gw_geo_df['GWEMONTH'].astype(str) + '-' + gw_geo_df['GWEDAY'].astype(str)
gw_geo_df['EndDate'] = pd.to_datetime(gw_geo_df['EndDate'])

need these lists to trim down the CoW and UCDP/PRIO conflict datasets

In [9]:
cshapes_cowcodes = cow_geo_df['COWCODE'].unique().tolist()
cshapes_gwcodes = gw_geo_df['GWCODE'].unique().tolist()

let's see how many countries change borders and how often

In [35]:
cow_geo_df.groupby('COWCODE').agg({'FEATUREID': 'nunique', 'CNTRY_NAME': lambda x: ', '.join(x), 'StartDate': 'min', 'EndDate': 'max'}).sort_values(by='FEATUREID', ascending=False).head(26)

,FEATUREID,CNTRY_NAME,StartDate,EndDate
COWCODE,,,,
345,6,"Yugoslavia, Yugoslavia, Serbia and Montenegro,...",1946-01-01,2016-06-30
651,5,"Egypt, Egypt, Egypt (United Arab Republic), Eg...",1946-01-01,2016-06-30
600,4,"Morocco, Morocco, Morocco, Morocco",1956-03-02,2016-06-30
850,4,"Indonesia, Indonesia, Indonesia, Indonesia",1949-12-27,2016-06-30
820,3,"Malaysia, Malaysia, Malaysia",1957-08-31,2016-06-30
770,3,"Pakistan, Pakistan, Pakistan",1947-08-14,2016-06-30
365,3,"Russia, USSR, USSR",1946-01-01,2016-06-30
435,3,"Mauritania, Mauritania, Mauritania",1960-11-28,2016-06-30
510,3,"Tanzania, Tanzania, Tanzania",1961-12-09,2016-06-30


In [37]:
gw_geo_df.groupby('GWCODE').agg({'FEATUREID': 'nunique', 'CNTRY_NAME': lambda x: ', '.join(x), 'StartDate': 'min', 'EndDate': 'max'}).sort_values(by='FEATUREID', ascending=False).head(26)

,FEATUREID,CNTRY_NAME,StartDate,EndDate
GWCODE,,,,
365,10,"Russia, USSR, USSR, USSR, USSR, USSR, USSR, US...",1946-01-01,2016-06-30
850,4,"Indonesia, Indonesia, Indonesia, Indonesia",1946-01-01,2016-06-30
600,4,"Morocco, Morocco, Morocco, Morocco",1956-03-02,2016-06-30
510,3,"Tanzania, Tanzania, Tanzania",1961-12-09,2016-06-30
345,3,"Yugoslavia, Serbia and Montenegro, Yugoslavia",1946-01-01,2006-06-02
770,3,"Pakistan, Pakistan, Pakistan",1947-08-14,2016-06-30
435,3,"Mauritania, Mauritania, Mauritania",1960-11-28,2016-06-30
820,3,"Malaysia, Malaysia, Malaysia",1957-08-31,2016-06-30
651,3,"Egypt, Egypt, Egypt",1946-01-01,2016-06-30


code citation: https://stackoverflow.com/questions/42151886/expanding-pandas-data-frame-with-date-range-in-columns
for expanding to a time-series from a time range

In [39]:
cow_geo_ts_df = pd.concat([pd.DataFrame({'month': pd.date_range(row.StartDate, row.EndDate, freq='MS'),
                                         'cow_id': row.COWCODE,
                                         'gw_id': row.GWCODE,
                                         'iso_alpha3': row.ISO1AL3,
                                         'country': row.CNTRY_NAME, 
                                         'FEATUREID': row.FEATUREID}, 
                                columns=['month', 'cow_id', 'iso_alpha3', 'gw_id', 'country', 'FEATUREID']) 
                           for i, row in cow_geo_df.iterrows()], ignore_index=True)
# note: this is not a geopandas df anymore
cow_geo_ts_df

,month,cow_id,iso_alpha3,gw_id,country,FEATUREID
0,1966-06-01,110,GUY,110,Guyana,0
1,1966-07-01,110,GUY,110,Guyana,0
2,1966-08-01,110,GUY,110,Guyana,0
3,1966-09-01,110,GUY,110,Guyana,0
4,1966-10-01,110,GUY,110,Guyana,0
...,...,...,...,...,...,...
123926,2016-02-01,626,SSD,626,South Sudan,246
123927,2016-03-01,626,SSD,626,South Sudan,246
123928,2016-04-01,626,SSD,626,South Sudan,246
123929,2016-05-01,626,SSD,626,South Sudan,246


In [40]:
gw_geo_ts_df = pd.concat([pd.DataFrame({'month': pd.date_range(row.StartDate, row.EndDate, freq='MS'),
                                        'gw_id': row.GWCODE,
                                        'cow_id': row.COWCODE,
                                        'iso_alpha3': row.ISO1AL3,
                                        'country': row.CNTRY_NAME, 
                                        'FEATUREID': row.FEATUREID}, 
                                columns=['month', 'gw_id', 'iso_alpha3', 'cow_id', 'country', 'FEATUREID']) 
                           for i, row in gw_geo_df.iterrows()], ignore_index=True)
# note: this is not a geopandas df anymore
gw_geo_ts_df

,month,gw_id,iso_alpha3,cow_id,country,FEATUREID
0,1966-06-01,110,GUY,110,Guyana,0
1,1966-07-01,110,GUY,110,Guyana,0
2,1966-08-01,110,GUY,110,Guyana,0
3,1966-09-01,110,GUY,110,Guyana,0
4,1966-10-01,110,GUY,110,Guyana,0
...,...,...,...,...,...,...
128757,1992-04-01,345,YUG,-1,Yugoslavia,247
128758,1991-09-01,365,SUN,-1,USSR,251
128759,1991-10-01,365,SUN,-1,USSR,253
128760,1991-11-01,365,SUN,-1,USSR,254


## Import CoW conflict data

In [41]:
cow_par = pd.read_csv("./Data/CorrelatesOfWar/war_participants.csv")
cow_con = pd.read_csv("./Data/CorrelatesOfWar/wars.csv")

In [42]:
cow_par['StartDate'] = pd.to_datetime(cow_par['StartDate'])
cow_par['EndDate'] = pd.to_datetime(cow_par['EndDate'])

In [44]:
cow_par = cow_par[cow_par['PolityID'].isin(cshapes_cowcodes)]
cow_cut = cow_par[(cow_par['StartDate'] >= '1945-01-01') & (cow_par['StartDate'] <= '2007-12-31')]
cow_merged = cow_cut.merge(cow_con[['WarID', 'WarLongName', 'WarTypeName']], how='left', on=['WarID'])
cow_merged['EndDate'] = cow_merged['EndDate'].fillna('2007-12-31')
cow_merged

,WarID,PolityID,StartDate,StartDate_Prec,EndDate,EndDate_Prec,Side,IsInitiator,Outcome,Deaths,WarLongName,WarTypeName
0,139,712,1945-08-10,Day,1945-08-14 00:00:00,Day,A,0,1,3000.0,World War II of 1939-1945,Inter-State War
1,139,365,1945-08-08,Day,1945-08-14 00:00:00,Day,A,0,1,7500000.0,World War II of 1939-1945,Inter-State War
2,147,750,1947-10-26,Day,1949-01-01 00:00:00,Day,A,1,6,2500.0,First Kashmir War of 1947-1949,Inter-State War
3,147,770,1947-10-26,Day,1949-01-01 00:00:00,Day,B,0,6,1000.0,First Kashmir War of 1947-1949,Inter-State War
4,148,666,1948-05-15,Day,1948-07-18 00:00:00,Day,A,0,1,3000.0,Arab-Israeli War of 1948-1949,Inter-State War
...,...,...,...,...,...,...,...,...,...,...,...,...
446,938,531,2006-10-19,Day,2008-06-11 00:00:00,Day,B,0,2,NaN,Third Somalia War of 2006-2008,Intra-State War
447,940,780,2006-10-11,Day,2007-12-31,Ongoing,A,1,5,NaN,Second Sri Lanka Tamil War of 2006-present,Intra-State War
448,941,679,2007-01-29,Day,2007-06-16 00:00:00,Day,A,0,6,1500.0,Second Yemeni Cleric War of 2007,Intra-State War
449,1573,732,1948-04-03,Day,1949-05-01 00:00:00,Month,B,0,1,NaN,Cheju Rebellion of 1948-1949,Non-State War


In [45]:
cow_war_ts = pd.concat([pd.DataFrame({'month': pd.date_range(row.StartDate, row.EndDate, freq='MS'),
                                        'cow_id': row.PolityID,
                                        'WarID': row.WarID,
                                        'WarType': row.WarTypeName}, 
                                columns=['month', 'cow_id', 'WarID', 'WarType']) 
                           for i, row in cow_merged.iterrows()], ignore_index=True)
cow_war_ts = cow_war_ts[cow_war_ts['month'] <= '2007-12-31']
cow_war_ts

,month,cow_id,WarID,WarType
0,1947-11-01,750,147,Inter-State War
1,1947-12-01,750,147,Inter-State War
2,1948-01-01,750,147,Inter-State War
3,1948-02-01,750,147,Inter-State War
4,1948-03-01,750,147,Inter-State War
...,...,...,...,...
13308,1971-06-01,698,1577,Non-State War
13309,1971-07-01,698,1577,Non-State War
13310,1971-08-01,698,1577,Non-State War
13311,1971-09-01,698,1577,Non-State War


In [46]:
cow_war_ts_gb = cow_war_ts.groupby(['month', 'cow_id']).agg({'WarID': 'count'})
cow_war_ts_gb = cow_war_ts_gb.reset_index().rename(columns={'WarID':'war_count'})
cow_war_ts_gb

,month,cow_id,war_count
0,1945-06-01,290,1
1,1945-06-01,365,3
2,1945-07-01,290,1
3,1945-07-01,365,3
4,1945-08-01,290,1
...,...,...,...
11336,2007-12-01,645,1
11337,2007-12-01,700,1
11338,2007-12-01,732,1
11339,2007-12-01,780,1


In [47]:
cow_war_ts_gb['war_count'].value_counts()

1    9605
2    1518
3     218
Name: war_count, dtype: int64

## Import UCDP/PRIO conflict data

In [48]:
ucdp_par = pd.read_csv("./Data/UCDP-PRIO_ArmedConflict/participants_gw.csv")
ucdp_obs = pd.read_csv("./Data/UCDP-PRIO_ArmedConflict/observations.csv")
ucdp_con = pd.read_csv("./Data/UCDP-PRIO_ArmedConflict/conflicts.csv")
ucdp_eps = pd.read_csv("./Data/UCDP-PRIO_ArmedConflict/episodes.csv")

In [49]:
ucdp_merged = ucdp_par.merge(ucdp_obs[['obs_id', 'intensity_level', 'ep_id']], on=['obs_id']) \
                      .merge(ucdp_con[['conflict_id', 'type_of_conflict']], on=['conflict_id']) \
                      .merge(ucdp_eps[['ep_id', 'start_date2', 'ep_end_date']], on=['ep_id'])

ucdp_merged = ucdp_merged[ucdp_merged['gw_id'].isin(cshapes_gwcodes)]
ucdp_merged = ucdp_merged[ucdp_merged['intensity_level'] == 'War']
ucdp_merged = ucdp_merged[ucdp_merged['role'] == 'primary']

ucdp_merged['start_date2'] = pd.to_datetime(ucdp_merged['start_date2'])
ucdp_merged['ep_end_date'] = pd.to_datetime(ucdp_merged['ep_end_date'])
ucdp_merged['ep_end_date'] = ucdp_merged['ep_end_date'].fillna('2018-12-31')
ucdp_merged

,obs_id,gw_id,side,role,conflict_id,year,intensity_level,ep_id,type_of_conflict,start_date2,ep_end_date
0,200-1946,145,A,primary,200,1946,War,200-1,Internal,1946-07-21,1946-07-21 00:00:00
11,202-1946,710,A,primary,202,1946,War,202-1,Internal,1946-12-31,1949-12-08 00:00:00
12,202-1947,710,A,primary,202,1947,War,202-1,Internal,1946-12-31,1949-12-08 00:00:00
13,202-1948,710,A,primary,202,1948,War,202-1,Internal,1946-12-31,1949-12-08 00:00:00
14,202-1949,710,A,primary,202,1949,War,202-1,Internal,1946-12-31,1949-12-08 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
4304,13692-2001,2,B,primary,13692,2001,War,13692-1,Interstate,2001-10-07,2001-11-13 00:00:00
4305,13692-2001,200,B,primary,13692,2001,War,13692-1,Interstate,2001-10-07,2001-11-13 00:00:00
4307,13694-2016,620,A,primary,13694,2016,War,13694-1,Internal,2015-03-18,2018-12-31
4318,13902-2016,640,A,primary,13902,2016,War,13902-1,Internal,2015-11-14,2017-09-06 00:00:00


In [50]:
ucdp_war_ts = pd.concat([pd.DataFrame({'month': pd.date_range(row.start_date2, row.ep_end_date, freq='MS'),
                                        'gw_id': row.gw_id,
                                        'WarID': row.conflict_id,
                                        'WarType': row.type_of_conflict}, 
                                columns=['month', 'gw_id', 'WarID', 'WarType']) 
                           for i, row in ucdp_merged.iterrows()], ignore_index=True)
ucdp_war_ts = ucdp_war_ts[ucdp_war_ts['month'] <= '2018-12-31']
ucdp_war_ts = ucdp_war_ts.drop_duplicates()
ucdp_war_ts

,month,gw_id,WarID,WarType
0,1947-01-01,710,202,Internal
1,1947-02-01,710,202,Internal
2,1947-03-01,710,202,Internal
3,1947-04-01,710,202,Internal
4,1947-05-01,710,202,Internal
...,...,...,...,...
120911,2018-08-01,840,14275,Internal
120912,2018-09-01,840,14275,Internal
120913,2018-10-01,840,14275,Internal
120914,2018-11-01,840,14275,Internal


In [51]:
ucdp_war_ts_gb = ucdp_war_ts.groupby(['month', 'gw_id']).agg({'WarID': 'count'})
ucdp_war_ts_gb = ucdp_war_ts_gb.reset_index().rename(columns={'WarID':'war_count'})
ucdp_war_ts_gb

,month,gw_id,war_count
0,1946-04-01,350,1
1,1946-05-01,350,1
2,1946-06-01,350,1
3,1946-07-01,350,1
4,1946-07-01,365,1
...,...,...,...
12372,2018-12-01,678,1
12373,2018-12-01,700,2
12374,2018-12-01,750,1
12375,2018-12-01,770,1


In [52]:
ucdp_war_ts_gb['war_count'].value_counts()

1    10665
2     1249
3      325
4      138
Name: war_count, dtype: int64

## Merge conflict data with geospatial data

In [25]:
cow_final_df = cow_geo_ts_df.merge(cow_war_ts_gb, on=['cow_id', 'month'], how='left')
cow_final_df['war_count'] = cow_final_df['war_count'].fillna(0).astype('int')
cow_final_gdf = cow_geo_df[['FEATUREID', 'geometry']].merge(cow_final_df, left_on=['FEATUREID'], right_on=['FEATUREID'])
cow_final_gdf

,FEATUREID,geometry,month,cow_id,iso_alpha3,country,war_count
0,0,"POLYGON ((-58.17262 6.81222, -58.15494 6.82819...",1966-06-01,110,GUY,Guyana,0
1,0,"POLYGON ((-58.17262 6.81222, -58.15494 6.82819...",1966-07-01,110,GUY,Guyana,0
2,0,"POLYGON ((-58.17262 6.81222, -58.15494 6.82819...",1966-08-01,110,GUY,Guyana,0
3,0,"POLYGON ((-58.17262 6.81222, -58.15494 6.82819...",1966-09-01,110,GUY,Guyana,0
4,0,"POLYGON ((-58.17262 6.81222, -58.15494 6.82819...",1966-10-01,110,GUY,Guyana,0
...,...,...,...,...,...,...,...
123926,246,"POLYGON ((34.09223 9.47747, 34.09416 9.45472, ...",2016-02-01,626,SSD,South Sudan,0
123927,246,"POLYGON ((34.09223 9.47747, 34.09416 9.45472, ...",2016-03-01,626,SSD,South Sudan,0
123928,246,"POLYGON ((34.09223 9.47747, 34.09416 9.45472, ...",2016-04-01,626,SSD,South Sudan,0
123929,246,"POLYGON ((34.09223 9.47747, 34.09416 9.45472, ...",2016-05-01,626,SSD,South Sudan,0


In [26]:
ucdp_final_df = gw_geo_ts_df.merge(ucdp_war_ts_gb, on=['gw_id', 'month'], how='left')
ucdp_final_df['war_count'] = ucdp_final_df['war_count'].fillna(0).astype('int')
ucdp_final_gdf = gw_geo_df[['FEATUREID', 'geometry']].merge(ucdp_final_df, left_on=['FEATUREID'], right_on=['FEATUREID'])
ucdp_final_gdf

,FEATUREID,geometry,month,gw_id,iso_alpha3,country,war_count
0,0,"POLYGON ((-58.173 6.812, -58.155 6.828, -58.03...",1966-06-01,110,GUY,Guyana,0
1,0,"POLYGON ((-58.173 6.812, -58.155 6.828, -58.03...",1966-07-01,110,GUY,Guyana,0
2,0,"POLYGON ((-58.173 6.812, -58.155 6.828, -58.03...",1966-08-01,110,GUY,Guyana,0
3,0,"POLYGON ((-58.173 6.812, -58.155 6.828, -58.03...",1966-09-01,110,GUY,Guyana,0
4,0,"POLYGON ((-58.173 6.812, -58.155 6.828, -58.03...",1966-10-01,110,GUY,Guyana,0
...,...,...,...,...,...,...,...
128757,247,"MULTIPOLYGON (((17.746 42.693, 17.740 42.692, ...",1992-04-01,345,YUG,Yugoslavia,1
128758,251,"MULTIPOLYGON (((73.656 39.455, 73.669 39.365, ...",1991-09-01,365,SUN,USSR,0
128759,253,"MULTIPOLYGON (((152.221 47.173, 152.207 47.125...",1991-10-01,365,SUN,USSR,0
128760,254,"MULTIPOLYGON (((152.221 47.173, 152.207 47.125...",1991-11-01,365,SUN,USSR,0


In [27]:
#type(ucdp_final_gdf)

In [28]:
#cow_final_df.to_csv("./Data/processed/cow_war_ts.json", index=False, encoding='utf-8')
#ucdp_final_df.to_csv("./Data/processed/ucdp_war_ts.json", index=False, encoding='utf-8')

In [33]:
cow_final_gdf_trimmed = cow_final_gdf[cow_final_gdf['month'] >= '2010-01-01']
cow_final_gdf_trimmed

,FEATUREID,geometry,month,cow_id,iso_alpha3,country,war_count
523,0,"POLYGON ((-58.17262 6.81222, -58.15494 6.82819...",2010-01-01,110,GUY,Guyana,0
524,0,"POLYGON ((-58.17262 6.81222, -58.15494 6.82819...",2010-02-01,110,GUY,Guyana,0
525,0,"POLYGON ((-58.17262 6.81222, -58.15494 6.82819...",2010-03-01,110,GUY,Guyana,0
526,0,"POLYGON ((-58.17262 6.81222, -58.15494 6.82819...",2010-04-01,110,GUY,Guyana,0
527,0,"POLYGON ((-58.17262 6.81222, -58.15494 6.82819...",2010-05-01,110,GUY,Guyana,0
...,...,...,...,...,...,...,...
123926,246,"POLYGON ((34.09223 9.47747, 34.09416 9.45472, ...",2016-02-01,626,SSD,South Sudan,0
123927,246,"POLYGON ((34.09223 9.47747, 34.09416 9.45472, ...",2016-03-01,626,SSD,South Sudan,0
123928,246,"POLYGON ((34.09223 9.47747, 34.09416 9.45472, ...",2016-04-01,626,SSD,South Sudan,0
123929,246,"POLYGON ((34.09223 9.47747, 34.09416 9.45472, ...",2016-05-01,626,SSD,South Sudan,0


## make the maps

In [ ]:
cow_map = KeplerGl(height=600)
cow_map.add_data(data=cow_final_gdf_trimmed, name="CoW Wars")
cow_map